### HuggingFace | BERT | PyTorch
- [Code Source](https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/)

In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f' Computations will utilize: {device}')
print('#####  Inital Program Setup is Complete  #####')

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
 Computations will utilize: cpu
#####  Inital Program Setup is Complete  #####


In [3]:
# Import data
df = pd.read_csv(r'C:\Data Sciences\Data\all_data_files\reviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17016 entries, 0 to 17015
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              17016 non-null  object
 1   userName              17016 non-null  object
 2   userImage             17016 non-null  object
 3   content               17016 non-null  object
 4   score                 17016 non-null  int64 
 5   thumbsUpCount         17016 non-null  int64 
 6   reviewCreatedVersion  14270 non-null  object
 7   at                    17016 non-null  object
 8   replyContent          9046 non-null   object
 9   repliedAt             9046 non-null   object
 10  sortOrder             17016 non-null  object
 11  appId                 17016 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.6+ MB


In [4]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOGxyMqOStnhbQ_mLfnLUfd1DHAt5uRXqDNArML...,Through Genesis,https://play-lh.googleusercontent.com/a/AATXAJ...,Redoing my review after messaging with the app...,1,40,5.15.3.0,2022-01-22 13:34:47,"Any.do is a free app, 90% of the app is comple...",2022-01-24 09:58:12,most_relevant,com.anydo
1,gp:AOqpTOH85sc18Ajgcgj6-IGmA7Gp34fVsrbyBJ274IZ...,Anthony Duarte,https://play-lh.googleusercontent.com/a-/AOh14...,"As a programmer, this is absolutely the most l...",1,38,5.15.3.0,2022-01-22 11:33:48,We are sorry you feel this way! We encourage u...,2022-01-24 09:59:17,most_relevant,com.anydo
2,gp:AOqpTOHgR1qnD3AZbHvKJ6-BbO4pMkokJS2JT1UvdVI...,Glenn Anderson,https://play-lh.googleusercontent.com/a/AATXAJ...,EDITED. So the app works as expected. It does ...,1,12,5.15.3.0,2022-01-22 00:29:36,For any questions feel free to contact us at w...,2022-01-11 15:51:10,most_relevant,com.anydo
3,gp:AOqpTOE3fQQpkWOMbSHW-DwukAnILBbMwBHEcbReiit...,Sydney Stoll,https://play-lh.googleusercontent.com/a/AATXAJ...,I have had this app for almost a year. I was r...,1,79,5.15.2.1,2021-11-07 15:58:27,This could happen if a user pressed on an adve...,2021-11-08 14:36:27,most_relevant,com.anydo
4,gp:AOqpTOH_GtAiezLqm0tHyquE1arU2C_L__IFTeqJxsg...,Bob Garcia,https://play-lh.googleusercontent.com/a-/AOh14...,I liked this app until recently. I discovered ...,1,78,5.15.3.0,2021-12-06 00:55:00,Sorry you feel this way Bob. As explained by o...,2021-12-06 12:36:53,most_relevant,com.anydo
